In [1]:
import numpy as np
import tensorflow as tf
import keras

path_to_file = keras.utils.get_file(
        'shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
        )
print(path_to_file)

with open(path_to_file) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

shakespeare_tensor = tf.constant([shakespeare_text])
vocab = list(set(shakespeare_text.lower().strip()))
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(shakespeare_text.lower())
n_tokens = len(tokenizer.word_index)

/Users/thomasbayer/.keras/datasets/shakespeare.txt
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



2025-04-11 09:25:56.146721: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2025-04-11 09:25:56.146738: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-04-11 09:25:56.146742: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
I0000 00:00:1744356356.146756 9259612 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1744356356.146781 9259612 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
from keras.models import load_model

# Load the model from the saved path
model = load_model("models/shakespeare_gru_1.keras")
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, None, 128)      │        64,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209,783 (819.47 KB)

 Trainable params: 69,927 (273.15 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 139,856 (546.32 KB)

In [24]:
text = "to be "
sequence = np.array(tokenizer.texts_to_sequences([text])[0])
print(sequence)
print(tokenizer.sequences_to_texts([sequence]))
sequence = sequence - 1
print(sequence)

[ 3  4  1 22  2  1]
['t o   b e  ']
[ 2  3  0 21  1  0]


In [11]:
inp = tf.one_hot(sequence, depth=n_tokens)
print(inp.shape)
inp_tf = tf.reshape(inp, (1, 6, n_tokens))
print(inp_tf.shape)


(6, 39)
(1, 6, 39)


In [16]:
y = model.predict(inp_tf)
print(y.shape)
prediction = y[0, -1, :]
print(prediction.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
(1, 6, 39)
(39,)


In [25]:
char_id = np.argmax(prediction) + 1
print(char_id)

sequence = np.append(sequence + 1, char_id)
print(sequence)
tokenizer.sequences_to_texts([sequence])

[ 3  4  1 22  2  1  5]


['t o   b e   a']

In [28]:
def next_token(sequence, model, temperature=1, transform=None, argmax=True):
    inp = transform(sequence) if transform else sequence


one_hot_fn = lambda x: tf.one_hot(x, depth=n_tokens)
sequence = np.array(tokenizer.texts_to_sequences([text])[0])
sequence = sequence - 1
next_token(sequence, model, transform=one_hot_fn)